[View in Colaboratory](https://colab.research.google.com/github/ruxandraburtica/recommender-systems/blob/master/1_memory_based_collaborative_filtering.ipynb)

# 1. Memory-based collaborative filtering


>[Memory-based collaborative filtering](#scrollTo=avRbNXr3L3k4)

>[Memory-based collaborative filtering](#scrollTo=4amzxC539k88)

>>>[When do you use collaborative filtering?](#scrollTo=4amzxC539k88)

>>>[Types of collaborative filtering](#scrollTo=4amzxC539k88)

>[Steps in building our collaborative filtering model:](#scrollTo=LmyTNkPNT4Uw)

>>[Steps that we need to additionally run, in order to set up our environment:](#scrollTo=LmyTNkPNT4Uw)

>>[Set up the environment](#scrollTo=9Zpoz98F81UV)

>>>[0.1. Get additional packages](#scrollTo=YUMd8nTW-sjD)

>>>[0.2. Import packages needed throughout the notebook](#scrollTo=Y3-_67QY9OoU)

>>>[0.3. Get the data](#scrollTo=XHKSiVyO-wvq)

>>[Metrics](#scrollTo=OxMMw3xxKndS)

>>[Compute prediction of rating](#scrollTo=iJCVCvaoUoAP)

>>>[2.1. Similarity measures](#scrollTo=iJCVCvaoUoAP)

>>>[Similarities](#scrollTo=OPPw0bMcEa9j)

>>>[2.2. Prediction:](#scrollTo=YtH57xKalh8W)

>>>[Prediction](#scrollTo=ysJ7OhCIkrzm)

>>>[Evaluate the model using cross-validation](#scrollTo=n-b2aFPo6kTK)

>>[Return top k products, similar to a given one](#scrollTo=2p-tDl8NA-Um)

>>>[Note for the trainers: I was thinking this part to be hands-on for participants](#scrollTo=2p-tDl8NA-Um)

>[Challenges in building a user-based collaborative filtering](#scrollTo=H29EP6EUIZAs)




# Memory-based collaborative filtering

Recommend items based only on past user's behaviour.


### When do you use collaborative filtering?
- you don't have to have any domain expertise in terms of what you're recommending (e.g. books, movies, music, car)
- leverage relationship between users

### Types of collaborative filtering
- user-based
- item-based


# Steps in building our collaborative filtering model:
1. Identify the metrics you want to use for the target user
2. For each product, compute a prediction of rating that the target user would give to the product
    - Identify the set of users most similar to the target user according to a similarity function (e.g. neighbourhood function)
    - Identify the products these similar users liked
    - For each of the products identified at the previous step, compute a prediction of rating that the target user would give to the product
3. Return top K products

## Steps that we need to additionally run, in order to set up our environment:

- Get additional packages (they can be installed in the colab environment)
- Imports 
- Get the data
- Analyze the data

## 0. Set up the environment

### 0.1. Get additional packages
 In Jupyter notebooks and colab, we can install additional packages

In [0]:
!pip uninstall -y scipy
!pip install scipy==1.0.0
!pip install surprise

**Note:** After installing, please restart the runtime (Runtime --> Restart runtime)


### 0.2. Import packages needed throughout the notebook

In [0]:
import requests
import zipfile
from io import BytesIO

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate, split

### 0.3. Get the data


We're going to use the MovieLens 100K dataset:
https://grouplens.org/datasets/movielens/100k/


We're going to download data ourselves, given that with your projects, probably data will not be available within `surprise`.

We are going to use the data in the `u.data` file that contains all the user-item ratings. 

In the u.data file each line represents a rating from a user to an item and the time when the rating happened. 

The format of each line is:
`userID itemID rating timestamp`, separated by tabs

In order to read the data, we're creating a Reader and define its format. In this case each line is divided as user item rating timestamp and is seperated by a tab \t. After we define the format we load our data in a Dataset object:

In [0]:
# Download archive and extract its contents.
ml_100k_url = 'http://files.grouplens.org/datasets/movielens/ml-100k.zip'
r = requests.get(ml_100k_url, stream=True)
z = zipfile.ZipFile(BytesIO(r.content))
z.extractall()

# Define the format
reader = Reader(line_format='user item rating timestamp', sep='\t')

# Load the data from the file using the reader format
data = Dataset.load_from_file('./ml-100k/u.data', reader=reader)

# Show first entries
print('{:>8} {:>8} {:>8} {:>8}'.format('user_id', 'item_id', 'rating', 'timestamp'))
for index, item in enumerate(data.raw_ratings):
  print('{:>8} {:>8} {:>8} {:>8}'.format(*item))
  if index > 5:
    break

## 1. Metrics

We're going to use the movie ratings as the metric for our collaborative filtering model.



Details about data contained in the zip archive can be found here:

http://files.grouplens.org/datasets/movielens/ml-100k/README

## 2. Compute prediction of rating


In oder to compute the similarity between users, most used methods are:
* **`Pearson Correlation`**  
* **`cosine similarity`**


We have the following as input data:
* A collection of users, $u_i$, $i=1, ...n$
* A collection of products $p_j$, $j=1, ...m$
* An $n$ x $m$ matrix of ratings $v_{ij}$, with $v_{ij}=?$ if user $i$ did not rate product $j$


### 2.1. Similarity measures

**Inner product:**

Basic similarity function:
$$inner(u_i, u_j) = \sum_{k=1}^{m} v_{ik} v_{jk}$$

**Cosine similarity:**

The inner product, however, is unbounded. 

One way to make it bounded between -1 and 1 is to divide by the vectors’ L2 norms, giving the cosine similarity:
$$cos(u_i, u_j) = \frac{\sum_{k=1}^{m} v_{ik} v_{jk}}{\sqrt{\sum_{k=1}^{m} v_{ik}^2\sum_{k=1}^{m} v_{jk}^2}}$$


**Pearson correlation:**

Cosine similarity is not invariant to shifts. What is invariant, though, is the Pearson correlation because it takes into account the mean of the vectors. Let $v_i$ and $v_k$ be the means of the two vectors:
 
$$u_{ik} = \frac{\sum_{j} (v_{ij} - v_{i})(v_{kj} - v_{k})}{\sqrt{\sum_{j} (v_{ij} - v_i)^2\sum_{j} (v_{kj} - v_k)^2}}$$


**Computing similarities**

|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 |
|---|---|---|---|---|---|---|---|
| A |4| | | 5 | 1 | | |
| B |5|5|4| | | | |
| C | | | |2|4|5| |
| D | |3 | | | | |3 |
| E | 5|2 | | |  | |  1|
| F |  |4 | | | 4 | |  3|


We can intuitively see that A and B have similar preferences, while A and C have different ones.
However, computing similarity by handling not rated movies with 0 does not reflect this difference that good.  Hence, we're going for pearson correlation, and the first step is computing the mean rating for each user: 

|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 | Mean |
|---|---|---|---|---|---|---|---|---|
| A |4| | | 5 | 1 | | |10/3|
| B |5|5|4| | | | |14/3|
| C | | | |2|4|5| |11/3|
| D | |3 | | | | |3 |3|
| E | 5|2 | | |  | |  1|8/3|
| F |  |4 | | | 4 | |  3|11/3|

Some people tend to be tough raters, while others tend to be easy raters. By substracting the mean of each row, we normalize the ratings, centering data around 0. This is why this is also called the centered cosine similarity.

|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 | Mean |
|---|---|---|---|---|---|---|---|---|
| A |2/3| | | 5/3 | -7/3 | | |10/3|
| B |1/3|1/3|-2/3| | | | |14/3|
| C | | | |-5/3|1/3|4/3| |11/3|
| D | |0 | | | | |0 |3|
| E | 7/3|-2/3 | | |  | |  -5/3|8/3|
| F |  |1/3 | | | 1/3 | |  -2/3|11/3|



### Similarities

Hands-on exercise: Compute the pearson correlation between user A and user B above

Answer: **0.09**

--- 

Individual exercise: Compute the pearson correlation between user A and user C above

Answer: **?**


### 2.2. Prediction:
We compute the prediction for user $i$ and product $j$ as a weighted average:

$$v_{ij}^\star = \frac{\sum_{v_{kj} \neq ?} u_{ik} v_{kj}}{\sum_{v_{kj}} u_{ik}}$$



Here, $u_{ik}$ is the similarity between user $i$ and user $k$ and $v_{k, j}$ is the rating that user $k$ gives to item $j$ with $v_{k, j} = ?$ if the user has not rated that item. 


**Compute predictions:**

Starting off from the previous matrix and having computed the similarity scores:


|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 | Similarity |
|---|---|---|---|---|---|---|---|---|
| A |4| | | 5 | 1 | | |1|
| B |5|5|4| | | | |0.09|
| C | | | |2|4|5| ||
| D | |3 | | | | |3 ||
| E | 5|2 | | |  | |  1||
| F |  |4 | | | 4 | |  3||

Let's say that we're using 2 closest neighbours to do predictions. In this case, for A, we would choose E and B, because they have the highest similarity scores.


|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 | Similarity |
|---|---|---|---|---|---|---|---|---|
| A |4| 2.24| | 5 | 1 | | |1|
| B |5|5|4| | | | |0.09|
| C | | | |2|4|5| ||
| D | |3 | | | | |3 ||
| E | 5|2 | | |  | |  1||
| F |  |4 | | | 4 | |  3||

### Prediction

Compute predictions for the other items of user A and complete the table below:


|  | HP1 | HP2 | HP3 | TW | SW1 | SW2 | SW3 | Similarity |
|---|---|---|---|---|---|---|---|---|
| A |4| 2.24|  | 5 | 1 | |  |1|
| B |5|5|4| | | | |0.09|
| C | | | |2|4|5| ||
| D | |3 | | | | |3 ||
| E | 5|2 | | |  | |  1||
| F |  |4 | | | 4 | |  3||


In [0]:
# We're going to use `KNNBasic` collaborative filtering algorithm in order to build an user-based recommender system.
# We're going to use `pearson` correlation.

sim_options = {
    'name': 'pearson_baseline',
    'user_based': False
}

# Fit to trainset.
algo = KNNBasic(sim_options=sim_options)
trainset = data.build_full_trainset()
algo.fit(trainset)

# Predict the rating for a specific (user, item) pair.
# Note that we know the actual rating.
uid = str(196)
itemid = str(302)
actual_rating = 4
algo.predict(uid=uid, iid=itemid, r_ui=actual_rating, verbose=True)

### Evaluate the model using cross-validation

Our model will try to optimize predictions, in order to match as closely as possible the actual results.

As we're trying to predict the rating of a certain user-movie combination, we will compare that prediction to the actual rating. The difference between the actual and the predicted rating is measured using classical error measurements such as Root mean squared error (RMSE) and Mean absolute error (MAE):


$$RMSE = \sqrt{\sum_{t=1}^{T}  {(\hat{y_t}-y_t)}^2 \over n}$$

$$MAE = {\sum_{t=1}^{T}  (\hat{y_t}-y_t) \over n}$$

Here, $\hat{y_t}$ is the prediction of $y_t$.

We're going to use collaborative filtering to predict what rating would a `(user, movie)` parent have, given all the other ratints. We're going to do that using SVD.

In [0]:
kf = split.KFold(random_state=0, n_splits=5)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=kf, verbose=True)

## 3. Return top k products, similar to a given one


In oder to compute the similarity between users, most used methods are **`Pearson Correlation`** and **`cosine similarity`**. 


In [0]:
import io
from surprise import get_dataset_dir

k = 5

def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = './ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid
  
  
# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the movie Toy Story
toy_story_raw_id = name_to_rid['Toy Story (1995)']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve neighbour ids of the nearest neighbors of Toy Story.
# Note: You can use the get_neighbours method:
# http://surprise.readthedocs.io/en/stable/algobase.html#surprise.prediction_algorithms.algo_base.AlgoBase.get_neighbors

# We need the neoighbour names, so we're going to use this part to convert from IDs to movie names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

print('The {} nearest neighbors of Toy Story are:'.format(k))
for movie in toy_story_neighbors:
    print(movie)

# Challenges in building a user-based collaborative filtering

* Sparsity of the (user, item) matrix 
    - large product sets, but users rank about 1% of the products
    - millions of users, and difficult to find users that have rated same movies
* Difficult to make predictions based on nearest-neighbours algorithms
* Scalability - the complexity grows with the number of products and users

**Next**: use latent models to capture similarity between users and items in a reduced dimensional space:
* Matrix factorization
* Clustering
* Projection (PCA)
